In [1]:
from bs4 import BeautifulSoup

In [64]:
# common util function
import re
def prettify_text(data):
    """Given a string, replace unicode chars and make it prettier"""

    # format it nicely: replace multiple spaces with just one
    data = re.sub(' +', ' ', data)
    # format it nicely: replace multiple new lines with just one
    data = re.sub('(\r?\n *)+', '\n', data)
    # format it nicely: replace bullet with *
    data = re.sub(u'\u2022', '* ', data)
    # format it nicely: replace registered symbol with (R)
    data = re.sub(u'\xae', ' (R) ', data)
    # format it nicely: remove trailing spaces
    data = data.strip('\'')
    # format it nicely: encode it, removing special symbols
    # data = data.encode('utf8', 'ignore') # Why is everything encoded???

    return str(data)

In [3]:
import requests
test_url = "https://www.apartments.com/la-scala-apartments-san-diego-ca/gpbynnw/" # From page 2

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
page = requests.get(test_url, headers=headers)

In [4]:
soup = BeautifulSoup(page.content, 'html.parser')
soup.prettify()

'<!DOCTYPE html>\n<html class="fontBaseSizeV2" data-placeholder-focus="false" id="appContainer" lang="en">\n <head>\n  <meta charset="utf-8"/>\n  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>\n  <meta content="telephone=no" name="format-detection"/>\n  <meta content="email=no" name="format-detection"/>\n  <meta content="true" name="HandheldFriendly"/>\n  <title>\n   La Scala Apartments - San Diego, CA | Apartments.com\n  </title>\n  <link href="/a/8a55f9/favicon.ico" rel="icon"/>\n  <link href="https://www.apartments.com/la-scala-apartments-san-diego-ca/gpbynnw/" rel="canonical">\n   <meta content="See all available apartments for rent at La Scala Apartments in San Diego, CA. La Scala Apartments has rental units ranging from 515-1010 sq ft starting at $2220. " name="description"/>\n   <meta content="en" name="language"/>\n   <meta content="en" http-equiv="language"/>\n   <meta content="en-US" http-equiv="content-language"/>\n   <meta content="739299276152207" 

In [82]:
def get_description(soup, fields):
    """Get the description for the apartment"""

    fields['description'] = ''

    if soup is None: return

    # find p with itemprop description
    obj = soup.find('section', {'class': 'descriptionSection'})

    if obj is not None:
        fields['description'] = prettify_text(obj.text)

In [83]:
fileds = {}
get_description(soup, fileds)

In [84]:
fileds

{'description': '\nAbout La Scala Apartments\nLa Scala offers luxury apartment villa living in a lush landscaped setting. Strategically located in the middle of San Diego in the beautiful community of University Towne Center UTC, La Scala is just a short walk away from amazing restaurants, entertainment, parks, and floorplans, one, and two bedroom apartments, spacious walk-in closets with patio and balcony areas that include a large storage area. Beautiful courtyards with peaceful Zen fountains, heated lap pool and Jacuzzi spa, and recently renovated BBQ areas. We have one of the best fitness centers with cardio machines, and exercise machines, plasma televisions, and free circuit training classes. You can enjoy our coffee bistro area located in the internet Lounge with WIFI access and PC as well as MAC computers’. In addition to all these great amenities, we also pay our water, sewer and trash utilities. We are open 7 days a week with a friendly office staff who can help with all your